In [1]:
import lusid
import lusid.models as models
import os
from msrest.authentication import BasicTokenAuthentication

api_url = os.getenv("FBN_LUSID_API_URL")
api_token = {"access_token": os.environ["ACCESS_TOKEN"]}
credentials = BasicTokenAuthentication(api_token)
client = lusid.LUSIDAPI(credentials, api_url)

In [2]:
import uuid
from datetime import datetime
import pytz
from collections import namedtuple
        
def build_transaction(trade_spec):
    return models.TransactionRequest(
        transaction_id=str(uuid.uuid4()),
        type="StockIn",
        instrument_uid=trade_spec.id,
        transaction_date=trade_spec.trade_date,
        settlement_date=trade_spec.trade_date,
        units=100,
        transaction_price=models.TransactionPrice(trade_spec.price),
        total_consideration=models.CurrencyAndAmount(100 * trade_spec.price, "GBP"),
        source="Client"
    )

In [3]:
# create the portfolio

scope = "finbourne"
guid = str(uuid.uuid4())
effective_date = datetime(2018, 1, 1, tzinfo=pytz.utc)

request = models.CreateTransactionPortfolioRequest(
    "portfolio-{0}".format(guid),
    "id-{0}".format(guid),
    base_currency="GBP",
    created=effective_date
)
result = client.create_portfolio(scope, request)
portfolio_id = result.id.code

In [4]:
# create instruments

instruments = [
    {"Figi": "BBG000C6K6G9", "Name": "VODAFONE GROUP PLC"},
    {"Figi": "BBG000C04D57", "Name": "BARCLAYS PLC"},
    {"Figi": "BBG000FV67Q4", "Name": "NATIONAL GRID PLC"},
    {"Figi": "BBG000BF0KW3", "Name": "SAINSBURY (J) PLC"},
    {"Figi": "BBG000BF4KL1", "Name": "TAYLOR WIMPEY PLC"}
]

figis_to_create = { i["Figi"]:models.InstrumentDefinition(i["Name"], {"Figi": i["Figi"]}) for i in instruments }

upsert_response = client.upsert_instruments(figis_to_create)

if len(upsert_response.failed) != 0:
    raise Exception(upsert_response.failed)

ids = client.get_instruments("Figi", [i["Figi"] for i in instruments])

instrument_ids = [i.lusid_instrument_id for i in ids.values.values()]

In [5]:
# add initial batch of trades

TransactionSpec = namedtuple('TradeSpec', 'id price trade_date')
trade_specs = [
    TransactionSpec(instrument_ids[0], 101, datetime(2018, 1, 1, tzinfo=pytz.utc)),
    TransactionSpec(instrument_ids[1], 102, datetime(2018, 1, 2, tzinfo=pytz.utc)),
    TransactionSpec(instrument_ids[2], 103, datetime(2018, 1, 3, tzinfo=pytz.utc))
]
trade_specs.sort(key=lambda ts: ts.id)

new_transactions = list(map(build_transaction, trade_specs))

# add initial batch of trades
initial_result = client.upsert_transactions(scope, portfolio_id, new_transactions)
as_at_batch1 = initial_result.version.as_at_date

In [6]:
# add trade for 2018-1-8
trade = build_transaction(TransactionSpec(instrument_ids[3], 104, datetime(2018, 1, 8, tzinfo=pytz.utc)))
later_trade = client.upsert_transactions(scope, portfolio_id, [trade])

as_at_batch2 = later_trade.version.as_at_date

In [7]:
# add back dated trade
trade = build_transaction(TransactionSpec(instrument_ids[4], 105, datetime(2018, 1, 5, tzinfo=pytz.utc)))
backdated_trade = client.upsert_transactions(scope, portfolio_id, [trade])

as_at_batch3 = backdated_trade.version.as_at_date

In [8]:
def print_transactions(transactions):
    for transaction in transactions:
        print("{0}\t{1}\t{2}\t{3}\t{4}".format(transaction.instrument_uid,
                                               transaction.transaction_date,
                                               transaction.units,
                                               transaction.transaction_price.price,
                                               transaction.total_consideration.amount))


In [9]:
# get the list of original trades
trades_list = client.get_transactions(scope, portfolio_id, as_at=as_at_batch1)

assert(len(trades_list.values) == 3)
print("transactions at {0}".format(as_at_batch1))
print_transactions(trades_list.values)

transactions at 2018-11-21 09:26:35.684010+00:00
LUID_CO3T26VB	2018-01-01 00:00:00+00:00	100.0	101.0	10100.0
LUID_1A6RYV70	2018-01-02 00:00:00+00:00	100.0	102.0	10200.0
LUID_3AIY3DZF	2018-01-03 00:00:00+00:00	100.0	103.0	10300.0


In [10]:
# get the list of trades including the later trade
trades_list = client.get_transactions(scope, portfolio_id, as_at=as_at_batch2)

assert(len(trades_list.values) == 4)
print("transactions at {0}".format(as_at_batch2))
print_transactions(trades_list.values)

transactions at 2018-11-21 09:26:36.723524+00:00
LUID_CO3T26VB	2018-01-01 00:00:00+00:00	100.0	101.0	10100.0
LUID_1A6RYV70	2018-01-02 00:00:00+00:00	100.0	102.0	10200.0
LUID_3AIY3DZF	2018-01-03 00:00:00+00:00	100.0	103.0	10300.0
LUID_OBAS87WL	2018-01-08 00:00:00+00:00	100.0	104.0	10400.0


In [11]:
# get the list of trades including the back-dated trade
trades_list = client.get_transactions(scope, portfolio_id, as_at=as_at_batch3)

assert(len(trades_list.values) == 5)
print("transactions at {0}".format(as_at_batch3))
print_transactions(trades_list.values)

transactions at 2018-11-21 09:26:38.497209+00:00
LUID_CO3T26VB	2018-01-01 00:00:00+00:00	100.0	101.0	10100.0
LUID_1A6RYV70	2018-01-02 00:00:00+00:00	100.0	102.0	10200.0
LUID_3AIY3DZF	2018-01-03 00:00:00+00:00	100.0	103.0	10300.0
LUID_EKHCMS76	2018-01-05 00:00:00+00:00	100.0	105.0	10500.0
LUID_OBAS87WL	2018-01-08 00:00:00+00:00	100.0	104.0	10400.0


In [12]:
# get the list of trades now
all_trades = client.get_transactions(scope, portfolio_id)
print("transactions at {0}".format(datetime.utcnow()))
print_transactions(all_trades.values)

transactions at 2018-11-21 09:26:46.350734
LUID_CO3T26VB	2018-01-01 00:00:00+00:00	100.0	101.0	10100.0
LUID_1A6RYV70	2018-01-02 00:00:00+00:00	100.0	102.0	10200.0
LUID_3AIY3DZF	2018-01-03 00:00:00+00:00	100.0	103.0	10300.0
LUID_EKHCMS76	2018-01-05 00:00:00+00:00	100.0	105.0	10500.0
LUID_OBAS87WL	2018-01-08 00:00:00+00:00	100.0	104.0	10400.0
